이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 12.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845493 sha256=9919685d1c51f1941d74a11599d1125fcb96030b1f7ad10026105052eee78648
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


**Spark Session**을 하나 만든다

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Spark FS Partition Demo") \
        .master("local[3]") \
        .enableHiveSupport() \
        .getOrCreate()

In [3]:
!wget https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv

--2024-06-20 05:39:03--  https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv
Resolving pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)... 52.92.152.154, 52.218.182.137, 52.92.234.74, ...
Connecting to pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)|52.92.152.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143130 (140K) [text/csv]
Saving to: ‘appl_stock.csv’

appl_stock.csv      100%[===================>] 139.78K  --.-KB/s    in 0.1s    

2024-06-20 05:39:04 (1.07 MB/s) - ‘appl_stock.csv’ saved [143130/143130]



In [4]:
df = spark.read.csv("appl_stock.csv", header=True, inferSchema=True)
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [5]:
df.show(10)

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [6]:
df = df.withColumn("year", year(df.Date)) \
    .withColumn("month", month(df.Date))

In [7]:
spark.sql("DROP TABLE IF EXISTS appl_stock")

DataFrame[]

In [8]:
df.write.partitionBy("year", "month").saveAsTable("appl_stock")

In [9]:
!ls -tl

total 156
drwxr-xr-x 3 root root   4096 Jun 20 05:40 spark-warehouse
drwxr-xr-x 5 root root   4096 Jun 20 05:40 metastore_db
-rw-r--r-- 1 root root    671 Jun 20 05:40 derby.log
drwxr-xr-x 1 root root   4096 Jun 14 17:39 sample_data
-rw-r--r-- 1 root root 143130 Jul  6  2018 appl_stock.csv


In [10]:
!ls -tl spark-warehouse/

total 4
drwxr-xr-x 9 root root 4096 Jun 20 05:40 appl_stock


In [11]:
!ls -tl spark-warehouse/appl_stock/

total 28
-rw-r--r--  1 root root    0 Jun 20 05:40  _SUCCESS
drwxr-xr-x 14 root root 4096 Jun 20 05:40 'year=2016'
drwxr-xr-x 14 root root 4096 Jun 20 05:40 'year=2015'
drwxr-xr-x 14 root root 4096 Jun 20 05:40 'year=2014'
drwxr-xr-x 14 root root 4096 Jun 20 05:40 'year=2013'
drwxr-xr-x 14 root root 4096 Jun 20 05:40 'year=2012'
drwxr-xr-x 14 root root 4096 Jun 20 05:40 'year=2011'
drwxr-xr-x 14 root root 4096 Jun 20 05:40 'year=2010'


In [12]:
!ls -tl spark-warehouse/appl_stock/year\=2010/

total 48
drwxr-xr-x 2 root root 4096 Jun 20 05:40 'month=12'
drwxr-xr-x 2 root root 4096 Jun 20 05:40 'month=11'
drwxr-xr-x 2 root root 4096 Jun 20 05:40 'month=10'
drwxr-xr-x 2 root root 4096 Jun 20 05:40 'month=9'
drwxr-xr-x 2 root root 4096 Jun 20 05:40 'month=8'
drwxr-xr-x 2 root root 4096 Jun 20 05:40 'month=7'
drwxr-xr-x 2 root root 4096 Jun 20 05:40 'month=6'
drwxr-xr-x 2 root root 4096 Jun 20 05:40 'month=5'
drwxr-xr-x 2 root root 4096 Jun 20 05:40 'month=4'
drwxr-xr-x 2 root root 4096 Jun 20 05:40 'month=3'
drwxr-xr-x 2 root root 4096 Jun 20 05:40 'month=2'
drwxr-xr-x 2 root root 4096 Jun 20 05:40 'month=1'


In [13]:
!ls -tl spark-warehouse/appl_stock/year\=2010/month\=12/

total 4
-rw-r--r-- 1 root root 3027 Jun 20 05:40 part-00000-3f0844ef-f09c-48bd-a3dc-10c84c5f75d2.c000.snappy.parquet


### How to Read from Partitioned table

In [14]:
df = spark.read.table("appl_stock").where("year = 2016 and month = 12")

In [15]:
df.show(10)

+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|               Date|              Open|              High|       Low|     Close|  Volume|         Adj Close|year|month|
+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|2016-12-01 00:00:00|        110.370003|        110.940002|109.029999|109.489998|37086900|        109.017344|2016|   12|
|2016-12-02 00:00:00|109.16999799999999|        110.089996|108.849998|109.900002|26528000|        109.425578|2016|   12|
|2016-12-05 00:00:00|             110.0|        110.029999|    108.25|109.110001|34324500|108.63898700000001|2016|   12|
|2016-12-06 00:00:00|             109.5|        110.360001|109.190002|109.949997|26195500|109.47535800000001|2016|   12|
|2016-12-07 00:00:00|        109.260002|        111.190002|109.160004|111.029999|29998700|        110.550697|2016|   12|
|2016-12-08 00:00:00|        110

In [16]:
spark.sql("SELECT * FROM appl_stock WHERE year = 2016 and month = 12").show(10)

+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|               Date|              Open|              High|       Low|     Close|  Volume|         Adj Close|year|month|
+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|2016-12-01 00:00:00|        110.370003|        110.940002|109.029999|109.489998|37086900|        109.017344|2016|   12|
|2016-12-02 00:00:00|109.16999799999999|        110.089996|108.849998|109.900002|26528000|        109.425578|2016|   12|
|2016-12-05 00:00:00|             110.0|        110.029999|    108.25|109.110001|34324500|108.63898700000001|2016|   12|
|2016-12-06 00:00:00|             109.5|        110.360001|109.190002|109.949997|26195500|109.47535800000001|2016|   12|
|2016-12-07 00:00:00|        109.260002|        111.190002|109.160004|111.029999|29998700|        110.550697|2016|   12|
|2016-12-08 00:00:00|        110